In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Hàm tải và chuẩn bị dữ liệu từ thư mục ảnh
def load_data(data_dir):
    images = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg"):
            label = filename.split("_")[0]
            if label in ["right", "forward", "left"]:
                img = cv2.imread(os.path.join(data_dir, filename))
                img = cv2.resize(img, (224, 224))  # Resize về kích thước 224x224
                images.append(img)
                labels.append(["right", "forward", "left"].index(label))
    return np.array(images), np.array(labels)

# Tải dữ liệu và chia tập huấn luyện/validation
data_dir = "dataset"  # Thay bằng đường dẫn thực tế đến thư mục chứa ảnh
images, labels = load_data(data_dir)
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu (MobileNetV2 yêu cầu giá trị pixel trong [-1, 1])
train_images = tf.keras.applications.mobilenet_v2.preprocess_input(train_images)
val_images = tf.keras.applications.mobilenet_v2.preprocess_input(val_images)

# Định nghĩa mô hình với MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Không huấn luyện lại MobileNetV2

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(3, activation="softmax")  # 3 lớp đầu ra: right, forward, left
])

# Compile mô hình
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Huấn luyện mô hình
history = model.fit(train_images, train_labels, epochs=50, validation_data=(val_images, val_labels))

# Vẽ đồ thị loss
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.savefig("loss_plot.png")  # Lưu đồ thị thành file
plt.close()

# Lưu mô hình dưới dạng .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("steering_model.tflite", "wb") as f:
    f.write(tflite_model)


Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 14s 675ms/step - accuracy: 0.5295 - loss: 1.0513 - val_accuracy: 0.5905 - val_loss: 0.9217
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 607ms/step - accuracy: 0.5834 - loss: 0.9446 - val_accuracy: 0.5524 - val_loss: 0.9372
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 599ms/step - accuracy: 0.5864 - loss: 0.9126 - val_accuracy: 0.5714 - val_loss: 0.9288
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 610ms/step - accuracy: 0.5996 - loss: 0.8902 - val_accuracy: 0.5905 - val_loss: 0.8776
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 589ms/step - accuracy: 0.6532 - loss: 0.8568 - val_accuracy: 0.6190 - val_loss: 0.9124
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 582ms/step - accuracy: 0.6400 - loss: 0.8759 - val_accuracy: 0.5714 - val_loss: 0.8704
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 580ms/step - accuracy: 0.6925 - loss: 0.7992 - val_accuracy: 0.5810 - val_loss: 0.9157
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 587ms/step - accuracy: 0.6427 - loss: 0.7964 - val_accuracy: 0

INFO:tensorflow:Assets written to: C:\Users\nguye\AppData\Local\Temp\tmprwion6kp\assets


Saved artifact at 'C:\Users\nguye\AppData\Local\Temp\tmprwion6kp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2210179606384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2210179602688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2210179606208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2210179602336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2210179604624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2210179612368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2210179608848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2210179608144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2210179609552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2210179611136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2

In [ ]:

# Tải mô hình .tflite
interpreter = tf.lite.Interpreter(model_path="steering_model.tflite")
interpreter.allocate_tensors()

# Lấy thông tin input và output của mô hình
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Kiểm tra trên luồng video
stream_url = "http://192.168.46.130:81/stream"
cap = cv2.VideoCapture(stream_url)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Không thể đọc luồng video.")
        break

    # Chuẩn bị frame cho dự đoán
    frame_resized = cv2.resize(frame, (224, 224))
    frame_processed = tf.keras.applications.mobilenet_v2.preprocess_input(frame_resized)
    input_data = np.expand_dims(frame_processed, axis=0).astype(np.float32)

    # Đặt dữ liệu đầu vào và chạy dự đoán
    interpreter.set_tensor(input_details[0]["index"], input_data)
    interpreter.invoke()

    # Lấy kết quả dự đoán
    output_data = interpreter.get_tensor(output_details[0]["index"])
    prediction = np.argmax(output_data[0])
    result = ["right", "forward", "left"][prediction]
    print(f"Dự đoán: {result}")

    # Hiển thị frame (tùy chọn)
    cv2.imshow("Video Stream", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):  # Nhấn 'q' để thoát
        break

# Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()